In [1]:
import SPiiPlusPython as sp 

In [39]:
# Open serial connection
hc = sp.OpenCommSerial(1, 115200)

In [148]:
# Running buffer 1 at the startup level for homing
sp.RunBuffer(hc, 1, "STARTUP", sp.SYNCHRONOUS, True)

In [48]:
# Enable X-axis
sp.Enable(hc, 0, sp.SYNCHRONOUS, True)
# Enable Y-axis
sp.Enable(hc, 1, sp.SYNCHRONOUS, True)

In [ ]:
# Moving axis to a point at a set velocity
sp.ExtToPoint(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    sp.Axis.ACSC_AXIS_0,
    # millimeters
    0,
    # millimeters per second
    velocity=15,
    wait=sp.SYNCHRONOUS,
    failure_check=True
)

In [106]:
# Moving multiple axes to a single point at specified velocity
sp.ExtToPointM(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY | sp.MotionFlags.ACSC_AMF_ENDVELOCITY,
    # Need the -1 at the end to mark the end of the axis array
    [0,1,-1],
    # X, Y points
    point=[-10,-10],
    velocity=50,
    endVelocity= 0,
    failure_check=True
)

In [ ]:
# Single axis job with specified velocity (Could set up with timing or external trigger?)
sp.Jog(
    hc, 
    sp.MotionFlags.ACSC_AMF_VELOCITY, 
    sp.Axis.ACSC_AXIS_1,
    # Direction and velocity (Positive or negative value)
    10,
    failure_check=True
    )

In [ ]:
# Single axis, multiple points

sp.MultiPoint(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    sp.Axis.ACSC_AXIS_1,
    1,
    failure_check=True,
)

# ExtAddPoint allows you to specify the values for the flags in the method, in this case, velocity
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 10, 25)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, -10, 15)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 15, 15)
sp.ExtAddPoint(hc, sp.Axis.ACSC_AXIS_1, 0, 15)


sp.EndSequence(hc, sp.Axis.ACSC_AXIS_1)


In [175]:
# Multi-axis, multiple points (Drawing a square)
axes = [0,1,-1]
sp.MultiPointM(
    hc,
    sp.MotionFlags.ACSC_AMF_VELOCITY,
    axes,
    1,
    failure_check=True
)

sp.ExtAddPointM(hc, axes, (0,0), 25)
for i in range(10):
    sp.ExtAddPointM(hc, axes, (-10,10), 25)
    sp.ExtAddPointM(hc, axes, (10,10), 25)
    sp.ExtAddPointM(hc, axes, (10,-10), 25)
    sp.ExtAddPointM(hc, axes, (-10,-10), 25)
sp.ExtAddPointM(hc, axes, (0,0), 25)
sp.EndSequenceM(hc, axes) # end of multipoint motion





In [171]:
sp.KillAll(hc)

In [22]:
# Disable all
sp.DisableAll(hc, sp.SYNCHRONOUS, True)

In [38]:
# Close connection
sp.CloseComm(hc)